In [2]:
pwd

'/root/eks-ml-pipeline'

In [3]:
!pip uninstall -y msspackages

Keyring is skipped due to an exception: 'keyring.backends'
Found existing installation: msspackages 0.0.7
Uninstalling msspackages-0.0.7:
  Successfully uninstalled msspackages-0.0.7


In [4]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl


In [5]:
from msspackages import setup_runner
setup_runner()

b"Hit:1 http://security.debian.org/debian-security buster/updates InRelease\nHit:2 http://deb.debian.org/debian buster InRelease\nHit:3 http://deb.debian.org/debian buster-updates InRelease\nReading package lists...\nBuilding dependency tree...\nReading state information...\n59 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nsudo is already the newest version (1.8.27-1+deb10u4).\n0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\ndefault-jre is already the newest version (2:1.11-71).\n0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.\nReading package lists...\nBuilding dependency tree...\nReading state information...\npython3 is already the newest version (3.7.3-1).\n0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.\nReading package lists...\nBuilding depend

In [6]:
!python3 -m pip install -r requirements.txt

Keyring is skipped due to an exception: 'keyring.backends'


<h1>Node, Pod & Container Training & Testing Data - Autoencoder<h1>

In [ ]:
%%time
from eks_ml_pipeline import node_autoencoder_ad_preprocessing, node_autoencoder_ad_feature_engineering, node_autoencoder_train_test_split

#pre processing
node_features_data, node_processed_data = node_autoencoder_ad_preprocessing("node_autoencoder_ad","v0.0.1","2022","10","10","10","128gb")

#test, train split
node_train_data, node_test_data = node_autoencoder_train_test_split(node_processed_data, [0.8,0.2])

#Train data feature engineering
node_training_data, node_training_tensor = node_autoencoder_ad_feature_engineering('train', node_features_data, node_train_data)

#Test data feature engineering
node_testing_data, node_testing_tensor = node_autoencoder_ad_feature_engineering('test', node_features_data, node_test_data)


Finished with sample #0
Finished with sample #1
Finished with sample #2
Finished with sample #3
Finished with sample #4
Finished with sample #5
Finished with sample #6
Finished with sample #7
Finished with sample #8
Finished with sample #9
Finished with sample #10
Finished with sample #11
Finished with sample #12
Finished with sample #13
Finished with sample #14
Finished with sample #15
Finished with sample #16
Finished with sample #17
Finished with sample #18
Finished with sample #19
Finished with sample #20
Finished with sample #21
Finished with sample #22
Finished with sample #23
Finished with sample #24
Finished with sample #25
Finished with sample #26
Finished with sample #27
Finished with sample #28
Finished with sample #29
Finished with sample #30
Finished with sample #31
Finished with sample #32
Finished with sample #33
Finished with sample #34
Finished with sample #35
Finished with sample #36
Finished with sample #37
Finished with sample #38
Finished with sample #39
Finished w

In [ ]:
%%time
from eks_ml_pipeline import pod_autoencoder_ad_preprocessing, pod_autoencoder_ad_feature_engineering, pod_autoencoder_train_test_split

#pre processing
pod_features_data, pod_processed_data = pod_autoencoder_ad_preprocessing("pod_autoencoder_ad","v0.0.1","2022","10","10","10","128gb")

#test, train split
pod_train_data, pod_test_data = pod_autoencoder_train_test_split(pod_processed_data,[0.8,0.2])

#Train data feature engineering
pod_training_data, pod_training_tensor = pod_autoencoder_ad_feature_engineering('train', pod_features_data, pod_train_data)

#Test data feature engineering
pod_testing_data, pod_testing_tensor = pod_autoencoder_ad_feature_engineering('test', pod_features_data, pod_test_data)


In [ ]:
%%time
from eks_ml_pipeline import container_autoencoder_ad_preprocessing, container_autoencoder_ad_feature_engineering, container_autoencoder_train_test_split

#pre processing
container_features_data, container_processed_data = container_autoencoder_ad_preprocessing("container_autoencoder_ad","v0.0.1","2022","10","10","10","128gb")

#test, train split
container_train_data, container_test_data = container_autoencoder_train_test_split(container_processed_data,[0.8,0.2])

#Train data feature engineering
container_training_data, container_training_tensor = container_autoencoder_ad_feature_engineering('train', container_features_data, container_train_data)

#Test data feature engineering
container_testing_data, container_testing_tensor = container_autoencoder_ad_feature_engineering('test', container_features_data, container_test_data)

<h1>Node, Pod & Container Training & Testing Data - PCA<h1>

In [ ]:
%%time
from eks_ml_pipeline import node_autoencoder_ad_preprocessing, node_autoencoder_ad_feature_engineering, node_autoencoder_train_test_split

#pre processing
node_features_data, node_processed_data = node_autoencoder_ad_preprocessing("node_pca_ad","v0.0.1","2022","10","10","10","128gb")

#test, train split
node_train_data, node_test_data = node_autoencoder_train_test_split(node_processed_data, [0.8,0.2])

#Train data feature engineering
node_training_data, node_training_tensor = node_autoencoder_ad_feature_engineering('train', node_features_data, node_train_data)

#Test data feature engineering
node_testing_data, node_testing_tensor = node_autoencoder_ad_feature_engineering('test', node_features_data, node_test_data)


In [ ]:
%%time
from eks_ml_pipeline import pod_autoencoder_ad_preprocessing, pod_autoencoder_ad_feature_engineering, pod_autoencoder_train_test_split

#pre processing
pod_features_data, pod_processed_data = pod_autoencoder_ad_preprocessing("pod_pca_ad","v0.0.1","2022","10","10","10","128gb")

#test, train split
pod_train_data, pod_test_data = pod_autoencoder_train_test_split(pod_processed_data,[0.8,0.2])

#Train data feature engineering
pod_training_data, pod_training_tensor = pod_autoencoder_ad_feature_engineering('train', pod_features_data, pod_train_data)

#Test data feature engineering
pod_testing_data, pod_testing_tensor = pod_autoencoder_ad_feature_engineering('test', pod_features_data, pod_test_data)


In [ ]:
%%time
from eks_ml_pipeline import container_autoencoder_ad_preprocessing, container_autoencoder_ad_feature_engineering, container_autoencoder_train_test_split

#pre processing
container_features_data, container_processed_data = container_autoencoder_ad_preprocessing("container_pca_ad","v0.0.1","2022","10","10","10","128gb")

#test, train split
container_train_data, container_test_data = container_autoencoder_train_test_split(container_processed_data,[0.8,0.2])

#Train data feature engineering
container_training_data, container_training_tensor = container_autoencoder_ad_feature_engineering('train', container_features_data, container_train_data)

#Test data feature engineering
container_testing_data, container_testing_tensor = container_autoencoder_ad_feature_engineering('test', container_features_data, container_test_data)

In [ ]:
#save tensor pod data
import numpy as np
np.savetxt('pod_training_tensor.txt', pod_training_tensor.reshape(pod_training_tensor.shape[0], -1))


In [ ]:
#read data from S3 bucket:
data_ingestion = Pyspark_data_ingestion()
spark = data_ingestion.get_spark()
pod_test_df = spark.read.parquet('s3a://emr-serverless-output-pd/aa/test_data1/')